### Imports

In [4]:
import pandas as pd
import numpy as np

from sklearn.dummy import  DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


### Data Loading

In [5]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

### Feature Engineering

In [0]:
SUFFIX_CAT = "_cat"
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values 

In [7]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df.price_value.values
  scores = cross_val_score(model, X, y, cv=3, scoring ='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

### Decision tree

In [9]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015792)

### Random Forrest

In [10]:
model = RandomForestRegressor(max_depth= 5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18718.657185256638, 64.5424578125788)

### XGboost

In [11]:
xgb_params = {
    'max_depth' : 5,
    'n_estimators' : 50,
    'learning_rate' : 0.1,
    'seed' :0 
}

run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[20:37:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:38:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:38:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [14]:
X = df[cat_feats].values
y = df.price_value.values

m = xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0).fit(X,y)
imp = PermutationImportance(m, random_state=0).fit(X,y)

[20:43:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [15]:
eli5.show_weights(imp, feature_names = cat_feats)

Weight,Feature
0.1194 ± 0.0031,param_napęd_cat
0.1132 ± 0.0032,param_rok-produkcji_cat
0.1090 ± 0.0025,param_stan_cat
0.0619 ± 0.0025,param_skrzynia-biegów_cat
0.0568 ± 0.0016,param_faktura-vat_cat
0.0489 ± 0.0014,param_moc_cat
0.0273 ± 0.0007,param_marka-pojazdu_cat
0.0242 ± 0.0013,feature_kamera-cofania_cat
0.0212 ± 0.0008,param_typ_cat
0.0174 ± 0.0008,param_pojemność-skokowa_cat


In [16]:
feats = ['param_napęd_cat',
'param_rok-produkcji_cat',
'param_stan_cat',
'param_skrzynia-biegów_cat',
'param_faktura-vat_cat',
'param_moc_cat',
'param_marka-pojazdu_cat',
'feature_kamera-cofania_cat',
'param_typ_cat',
'param_pojemność-skokowa_cat',
'seller_name_cat',
'feature_wspomaganie-kierownicy_cat',
'param_model-pojazdu_cat',
'param_wersja_cat',
'param_kod-silnika_cat',
'feature_system-start-stop_cat',
'feature_asystent-pasa-ruchu_cat',
'feature_czujniki-parkowania-przednie_cat',
'feature_łopatki-zmiany-biegów_cat',
'feature_regulowane-zawieszenie_cat']
len(feats)

20

In [17]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[20:49:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:49:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:49:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13375.230420852275, 65.40441107118909)

In [18]:
df['param_napęd'].unique()

array([None, 'Na przednie koła', '4x4 (dołączany automatycznie)',
       'Na tylne koła', '4x4 (dołączany ręcznie)', '4x4 (stały)'],
      dtype=object)

In [19]:
df['param_rok-produkcji'].unique()

array(['2018', '2011', '2015', '2009', '2017', '2012', '2013', '2007',
       '2001', '2016', '2006', '2008', '2004', '1999', '2000', '2010',
       '2005', '2002', '1998', '2014', '2003', '1982', '1995', '1997',
       '1992', '1993', '1994', '1996', '1989', '1988', '1967', '1987',
       '1970', '1959', '1990', '1991', '1974', None, '1975', '1973',
       '1953', '1985', '1984', '1986', '1981', '1979', '1960', '1983',
       '1978', '1964', '1980', '1972', '1969', '1956', '1966', '1977',
       '1962', '1965', '1971', '1963', '1961', '1952', '1949', '1976',
       '1937', '1968', '1958', '1955', '1933', '1929', '1957', '1944',
       '1954', '1932', '1936', '1947', '1948'], dtype=object)

In [20]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_rok-produkcji'].unique()

array([2018, 2011, 2015, 2009, 2017, 2012, 2013, 2007, 2001, 2016, 2006,
       2008, 2004, 1999, 2000, 2010, 2005, 2002, 1998, 2014, 2003, 1982,
       1995, 1997, 1992, 1993, 1994, 1996, 1989, 1988, 1967, 1987, 1970,
       1959, 1990, 1991, 1974,   -1, 1975, 1973, 1953, 1985, 1984, 1986,
       1981, 1979, 1960, 1983, 1978, 1964, 1980, 1972, 1969, 1956, 1966,
       1977, 1962, 1965, 1971, 1963, 1961, 1952, 1949, 1976, 1937, 1968,
       1958, 1955, 1933, 1929, 1957, 1944, 1954, 1932, 1936, 1947, 1948])

In [21]:
feats = ['param_napęd_cat',
'param_rok-produkcji',
'param_stan_cat',
'param_skrzynia-biegów_cat',
'param_faktura-vat_cat',
'param_moc_cat',
'param_marka-pojazdu_cat',
'feature_kamera-cofania_cat',
'param_typ_cat',
'param_pojemność-skokowa_cat',
'seller_name_cat',
'feature_wspomaganie-kierownicy_cat',
'param_model-pojazdu_cat',
'param_wersja_cat',
'param_kod-silnika_cat',
'feature_system-start-stop_cat',
'feature_asystent-pasa-ruchu_cat',
'feature_czujniki-parkowania-przednie_cat',
'feature_łopatki-zmiany-biegów_cat',
'feature_regulowane-zawieszenie_cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[20:49:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:49:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:50:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11308.885890938496, 27.868488259630677)

In [22]:
df.param_moc.unique()

array(['90 KM', '115 KM', '262 KM', '110 KM', '310 KM', '105 KM',
       '140 KM', '175 KM', '125 KM', '185 KM', '190 KM', '440 KM',
       '141 KM', '200 KM', '224 KM', '75 KM', '99 KM', '184 KM', '109 KM',
       '233 KM', '116 KM', '68 KM', '286 KM', '126 KM', '160 KM',
       '135 KM', '120 KM', '272 KM', None, '150 KM', '180 KM', '136 KM',
       '102 KM', '131 KM', '218 KM', '245 KM', '170 KM', '112 KM',
       '250 KM', '252 KM', '73 KM', '100 KM', '313 KM', '101 KM',
       '285 KM', '70 KM', '383 KM', '174 KM', '277 KM', '132 KM',
       '130 KM', '215 KM', '60 KM', '330 KM', '163 KM', '177 KM', '98 KM',
       '78 KM', '189 KM', '156 KM', '143 KM', '69 KM', '113 KM', '65 KM',
       '122 KM', '82 KM', '251 KM', '95 KM', '197 KM', '235 KM', '238 KM',
       '171 KM', '381 KM', '400 KM', '178 KM', '80 KM', '165 KM', '85 KM',
       '258 KM', '142 KM', '204 KM', '124 KM', '55 KM', '144 KM',
       '231 KM', '248 KM', '152 KM', '181 KM', '210 KM', '340 KM',
       '129 KM', '147 

In [24]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split('K')[0].replace(" ", "")))


AttributeError: ignored

In [25]:
df.param_moc.unique()

array([    90,    115,    262,    110,    310,    105,    140,    175,
          125,    185,    190,    440,    141,    200,    224,     75,
           99,    184,    109,    233,    116,     68,    286,    126,
          160,    135,    120,    272,     -1,    150,    180,    136,
          102,    131,    218,    245,    170,    112,    250,    252,
           73,    100,    313,    101,    285,     70,    383,    174,
          277,    132,    130,    215,     60,    330,    163,    177,
           98,     78,    189,    156,    143,     69,    113,     65,
          122,     82,    251,     95,    197,    235,    238,    171,
          381,    400,    178,     80,    165,     85,    258,    142,
          204,    124,     55,    144,    231,    248,    152,    181,
          210,    340,    129,    147,     50,     54,    290,    306,
          193,     77,    164,     96,    194,    111,    166,    206,
          118,    360,    211,    271,    455,    280,    106,    114,
      

In [26]:
feats = ['param_napęd_cat',
'param_rok-produkcji',
'param_stan_cat',
'param_skrzynia-biegów_cat',
'param_faktura-vat_cat',
'param_moc',
'param_marka-pojazdu_cat',
'feature_kamera-cofania_cat',
'param_typ_cat',
'param_pojemność-skokowa_cat',
'seller_name_cat',
'feature_wspomaganie-kierownicy_cat',
'param_model-pojazdu_cat',
'param_wersja_cat',
'param_kod-silnika_cat',
'feature_system-start-stop_cat',
'feature_asystent-pasa-ruchu_cat',
'feature_czujniki-parkowania-przednie_cat',
'feature_łopatki-zmiany-biegów_cat',
'feature_regulowane-zawieszenie_cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[20:54:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:54:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:54:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9754.814038220027, 78.76301552059402)

In [27]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(" ","")))

feats = ['param_napęd_cat',
'param_rok-produkcji',
'param_stan_cat',
'param_skrzynia-biegów_cat',
'param_faktura-vat_cat',
'param_moc',
'param_marka-pojazdu_cat',
'feature_kamera-cofania_cat',
'param_typ_cat',
'param_pojemność-skokowa',
'seller_name_cat',
'feature_wspomaganie-kierownicy_cat',
'param_model-pojazdu_cat',
'param_wersja_cat',
'param_kod-silnika_cat',
'feature_system-start-stop_cat',
'feature_asystent-pasa-ruchu_cat',
'feature_czujniki-parkowania-przednie_cat',
'feature_łopatki-zmiany-biegów_cat',
'feature_regulowane-zawieszenie_cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[20:54:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:54:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:54:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9621.119663721702, 100.59307679134791)